In [1]:
import pandas as pd
import numpy as np 
import math as m 

In [ ]:
train_df = pd.read_csv('train.csv',index_col='PassengerId')
test_df = pd.read_csv('test.csv',index_col='PassengerId')

In [3]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    f

In [4]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [5]:
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# Fill missing age values with the median age of passengers grouped by Pclass and Sex
train_df['Age'] = train_df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
test_df['Age'] = test_df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

In [7]:
train_df.fillna({'Cabin':'U'},inplace=True)
test_df.fillna({'Cabin':'U'},inplace=True)


In [8]:
# Fill missing 'Embarked' values with the most frequent value (mode)
train_df.fillna({'Embarked': train_df['Embarked'].mode()[0]}, inplace=True)

In [9]:
# fill the missing 'Fare' values in the test dataset with the median fare of passengers grouped by Pclass and Embarked
test_df['Fare'] = test_df.groupby(['Pclass', 'Embarked'])['Fare'].transform(lambda x: x.fillna(x.median()))

In [10]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     891 non-null    object 
 10  Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       418 non-null    f

In [11]:
def cabin_to_num(input : str):
    def name_to_num(name : str):
        if len(name) == 1:
            return (ord(name[0])- ord('A'))* 1000
        else:
            return (ord(name[0])- ord('A'))* 1000 + int(name[1:])
        
    cabins = input.split(' ')
    sum = 0 
    for cabin in cabins:
        if cabin == 'U':
            return -1
        else:
            sum += name_to_num(cabin)
    return sum/len(cabins)

In [12]:
# Convert 'Sex' column to numeric
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

train_df['Cabin'] = train_df['Cabin'].apply(cabin_to_num)
test_df['Cabin'] = test_df['Cabin'].apply(cabin_to_num)

# Convert 'Embarked' column to numeric
embarked_mapping = {'C': 0, 'Q': 1, 'S': 2}
train_df['Embarked'] = train_df['Embarked'].map(embarked_mapping)
test_df['Embarked'] = test_df['Embarked'].map(embarked_mapping)

# Since Tickets values have weird patterns, we will drop this column
train_df.drop('Ticket', axis=1, inplace=True)
test_df .drop('Ticket', axis=1, inplace=True)

# Drop the 'Name' column from the train and test datasets
train_df.drop('Name', axis=1, inplace=True)
test_df.drop('Name', axis=1, inplace=True)



# Display the first few rows of the modified train_ds and test_ds
test_df.head()
train_df.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,0,3,0,22.0,1,0,7.2500,-1.0,2
2,1,1,1,38.0,1,0,71.2833,2085.0,0
3,1,3,1,26.0,0,0,7.9250,-1.0,2
4,1,1,1,35.0,1,0,53.1000,2123.0,2
5,0,3,0,35.0,0,0,8.0500,-1.0,2


In [13]:
from sklearn.preprocessing import MinMaxScaler

# Select numeric columns
numeric_cols = ['Age', 'Fare', 'Pclass', 'SibSp', 'Parch', 'Cabin','Embarked']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the train dataset
train_df[numeric_cols] = scaler.fit_transform(train_df[numeric_cols])

# Transform the test dataset
test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])



In [22]:
# Display a random sample of 5 rows from the normalized test_ds
train_df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
369,1,1.0,1,0.264891,0.000,0.000000,0.015127,0.000000,0.5
402,0,1.0,0,0.321438,0.000,0.000000,0.015713,0.000000,1.0
660,0,0.0,0,0.723549,0.000,0.333333,0.221098,0.160465,0.0
435,0,0.0,0,0.623021,0.125,0.000000,0.109110,0.212884,1.0
620,0,0.5,0,0.321438,0.000,0.000000,0.020495,0.000000,1.0


In [15]:
#extract the frist 600 records for train
x_train = train_df[:600].drop('Survived', axis=1)
y_train = train_df[:600]['Survived']

#extract the last 200 records for test
x_test = train_df[-200:].drop('Survived', axis=1)
y_test = train_df[-200:]['Survived']

In [121]:
def Relu(x):
    return x if x>0 else 0

def d_Relu(x):
    return 1. if x>0 else 0

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def d_sigmoid(x):
    return x * (1 - x)

def d_tanh(x):
    return 1 - np.tanh(x)**2

In [128]:
w_1 = np.random.uniform(-1, 1, (8, 5))
b_1 = np.random.uniform(-1, 1, (5))
af1 = np.vectorize(Relu)
df1 = np.vectorize(d_Relu)
w_1

array([[-0.45605866, -0.62329823, -0.70916749,  0.01896665,  0.00209211],
       [-0.71988733, -0.2252022 ,  0.84989176, -0.7150477 ,  0.48632681],
       [ 0.6895161 , -0.1531751 ,  0.58607716,  0.31627141, -0.05011497],
       [-0.66539524, -0.16884315,  0.99103288,  0.39737356, -0.6998812 ],
       [-0.06964465, -0.63517465,  0.69582065, -0.0995512 , -0.8311853 ],
       [-0.05988939, -0.67493426, -0.26628252,  0.24902477,  0.96157744],
       [-0.23881424, -0.51286213,  0.43404304,  0.85356819,  0.43491128],
       [-0.46989873,  0.78805639,  0.14910333, -0.18944132,  0.61152541]])

In [97]:
w_2 = np.random.uniform(1,-1,(5,3))
b_2 = np.random.uniform(1,-1,(3))
af2 = np.tanh
df2 = d_tanh

In [98]:
w_3 = np.random.uniform(1,-1,(3,1))
b_3 = np.random.uniform(1,-1,(1))
af3 = sigmoid

In [133]:
def forward_prop(x):
    z1 = x @ w_1 
    z1 = z1 + b_1 
    a1 = af1(z1)
    print('a1 : \n' , a1)
    z2 = a1 @ w_2 + b_2
    print( 'z2: \n', z2)
    a2 = af2(z2)
    print('a2 :\n',a2)
    z3 = a2 @ w_3 + b_3
    print('z3: \n', z3)
    a3 = af3(z3)
    return a3


mx = np.random.randn(5, 8)
yx = (np.random.randn(5, 1) < 0 ).astype(np.float64)
print(mx)
print(yx)
forward_prop(mx)

[[ 0.24716609  0.44238908  1.62458732 -0.75261525  1.02487812 -0.34761868
   0.92864163  0.30900854]
 [ 0.0879879   0.28848621  0.03418569  0.30398259 -0.37789746 -0.08398693
   1.33522824  1.16212216]
 [ 1.24472476 -0.54170174 -0.61448422  0.71146631 -1.69642534  0.4786086
   0.73073163 -0.17494887]
 [-0.24896279  0.08946756 -1.99357453 -1.29498172 -0.67396117 -0.17253406
  -2.66909101 -1.98361752]
 [-1.59075007  0.7787089   2.11763435  0.46157589  0.64309054  0.79581055
   1.0829891  -0.0397269 ]]
[[1.]
 [1.]
 [0.]
 [1.]
 [1.]]
a1 : 
 [[0.16622399 0.         2.00448813 0.00934961 0.        ]
 [0.         0.         1.3589939  0.42397717 0.56509244]
 [0.         0.         0.         1.00498676 0.46745253]
 [0.55698006 0.4517361  0.         0.         0.        ]
 [0.3794762  0.         4.53084583 0.89299051 0.        ]]
z2: 
 [[-1.38484267 -0.45623683 -0.68683894]
 [-1.28228249 -0.7064918  -0.22844503]
 [-0.4357303   0.10039322  0.46714191]
 [ 0.16482819  0.89891315 -0.07890612]
 [-2

array([[0.25717521],
       [0.27647967],
       [0.57137738],
       [0.69212073],
       [0.21711938]])

In [ ]:
def back_prop(x, y,alpha):
    global w_1, w_2, w_3, b_1, b_2, b_3
    z1 = x @ w_1 + b_1
    a1 = af1(z1)
    z2 = a1 @ w_2 + b_2
    a2 = af2(z2)
    z3 = a2 @ w_3 + b_3
    a3 = af3(z3)
    print('a3: \n', a3)
    d3 = a3 - y
    dw3 = a2.T * d3
    print('d3: \n', d3 ,'dw3: \n',dw3)
    d2 = d3 @ w_3.T * d_sigmoid(a2)
    print('d2: \n', d2)
    d1 = d2 @ w_2.T * d_Relu(a1)
    print('d1: \n', d1)
    w_3 = w_3 - (alpha*dw3)
    b_3 = b_3 - np.sum(d3, axis=0)
    w_2 = w_2 - a1.T @ d2
    b_2 = b_2 - np.sum(d2, axis=0)
    w_1 = w_1 - x.T @ d1
    b_1 = b_1 - np.sum(d1, axis=0)

back_prop(mx, yx, 0.01)

a3: 
 [[0.25717521]
 [0.27647967]
 [0.57137738]
 [0.69212073]
 [0.21711938]]
a2: 
 [[-0.88203105 -0.42701217 -0.59594744]
 [-0.85709186 -0.60847222 -0.22455231]
 [-0.41009906  0.10005729  0.43588714]
 [ 0.16335153  0.71576825 -0.07874276]
 [-0.98996885 -0.90493395 -0.22698445]]
d3: 
 [[-0.74282479]
 [-0.72352033]
 [ 0.57137738]
 [-0.30787927]
 [-0.78288062]] dw3: 
 [[ 0.65519452  0.31719522  0.44268453]
 [ 0.62012338  0.44024202  0.16246816]
 [-0.23432133  0.05717047  0.24905606]
 [-0.05029255 -0.22037021  0.02424326]
 [ 0.77502743  0.70845526  0.17770173]]
d2: 
 [[ 0.74128047  0.3651451   0.43772162]
 [ 0.69230427  0.57123684  0.12326242]
 [-0.19863097  0.04150475  0.08704587]
 [-0.02529486 -0.05052853  0.01620295]
 [ 0.92714758  1.08868818  0.13508769]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()